<a href="https://colab.research.google.com/github/tomonari-masada/course2023-sml/blob/main/05_nearest_neighbors_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# k近傍法
* 直感的には、「どんな友人と付き合っているかを見ればその人が分かる」という考え方にもとづく分類および回帰の手法。
* 新しいデータ点について、対応するターゲットの値を予測する場合、そのデータ点のk個の近傍のクラスを元に予測する。
 * クラスを予測する（分類）の場合、予測は多数決で行われる。
 * 数値を予測する（回帰）の場合、近傍のターゲットの値の平均を予測値とする。（他の方法もありうる。）

## 5.1 準備

### 5.1.1 必要なパッケージのインポート

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'

### 5.1.2 データファイルのアップロード
* 特徴量は、「国民一人当たりのGDP (GDP per capita)」と「雇用率 (employment rate)」の、二つを使うことにする。
* ターゲットは、生活満足度。つまり・・・
* 「国民一人当たりのGDP (GDP per capita)」と「雇用率 (employment rate)」を使って、生活満足度を予測する。

* `lifesat_extended.csv`というファイルをColabにアップロードする。

In [ ]:
from google.colab import files
files.upload()

### 5.1.3 データファイルの内容を確認

In [ ]:
df = pd.read_csv('lifesat_extended.csv')

In [ ]:
df

* 国名をインデックスに設定してデータフレームを作り直す。

In [ ]:
df = df.set_index('Country')

* データフレームのさまざまな情報を確認する。

In [ ]:
df.info()

In [ ]:
df.describe()

* GDP per capitaやemployment rateを使って生活満足度を予測するという問題設定が、そもそも妥当な問題設定かどうか、確認する。
 * 特徴量とターゲットとのあいだに、ある程度の関連性がないと、解けない問題を解こうとしていることになる。

In [ ]:
df.plot(kind='scatter', x='GDP per capita', y='Life satisfaction');

In [ ]:
df.plot(kind='scatter', x='Employment rate', y='Life satisfaction');

## 5.2 問題設定
* 最終的に、日本の生活満足度を予測したい、とする。
 * もちろん、すでに答えは分かっているが・・・
 * ここでは、日本の生活満足度の予測を、最終的に解きたい問題だと設定する。
 * 現実には、本当に解きたい問題は、あらかじめ答えが分かっていない。
* 言い換えれば、日本のデータが、テストデータとなる。
 * テストデータ＝最終的に手法の性能を評価するために使うデータ集合。
 * テストデータでの評価結果を見て、手法のチューニングへ戻ってきてはいけない。
* そこで、日本を除外したデータフレームを作り、それを使う。

In [ ]:
df_test = df.loc[['Japan']]

In [ ]:
df_test

In [ ]:
df = df.drop(['Japan'])

In [ ]:
df

## 5.3 k近傍法の実装（k=1の場合）
* 例題として、韓国の生活満足度の予測を考える。
* そこで、最も韓国に近い国の生活満足度でもって、韓国の生活満足度の予測値とする手法を、実装してみる。
* 最も韓国に近い国を、どうやって選べばいいだろうか？

### 5.3.1 特徴量とターゲットの用意

* 特徴量である(一人当たりのGDP, 雇用率)と、ターゲットである生活満足度とを、別々に保存する。

In [ ]:
X = df[['GDP per capita', 'Employment rate']]
y = df['Life satisfaction']

In [ ]:
X

* 韓国の生活満足度を予測したい。
* そのために、韓国の近傍となる国を求める必要がある。
* 近傍を求めるには、特徴量の"近さ"を使う。

In [ ]:
X.loc['Korea']

* なお、以下のようにすれば、インデックス、つまり国名を枚挙できる。

In [ ]:
for country in df.index:
  print(country)

* 以下のようにすれば、各国の特徴量をNumpyの配列として枚挙できる。

In [ ]:
for country in df.index:
  print(X.loc[country].values)

* （雇用率は、しばらく使わないことにする。）

### 5.3.2 演習

* 演習問題5-1： 一人当たりのGDPが、韓国に最も近い国を求めてみよう。

* 演習問題5-2： 上で求めた国の生活満足度を、韓国の生活満足度の予測値としたとする。このとき、予測誤差はいくらになるか。

## 5.4 k近傍法の実装

### 5.4.1 演習

* 演習問題5-3： 韓国以外の全ての国について、一人当たりのGDPを使って計算される韓国との距離を、求めてみよう。

* 演習問題5-4：韓国の一人当たりのGDPに近い順に、他の全ての国を並べ替えて、表示させてみよう。

* 演習問題5-5：変数kに、求めるべき近傍の個数が下のようにセットされているとする。韓国のk個の近傍の国名を、韓国に近い順に表示させてみよう。

In [ ]:
k = 5


* 演習問題5-6：上で求めたk個の国について、それらの国の生活満足度の平均値を求めてみよう。

* 演習問題5-7：上の平均値を、韓国の生活満足度の予測値とした場合、予測誤差はいくらになるか。

# 課題5-1
* 変数kの値を、1から27まで変化させたとき、最も予測誤差が小さくなるのは、kの値がいくらのときか。

# 課題5-2
* 一人当たりのGDPを使うのではなく、雇用率を使って生活満足度の予測を行ったとき、最も予測誤差を小さくするkの値はいくらか。